In [26]:
import dill
dill.dump_session("EXP2.db")

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split



In [2]:
from keras.models import Sequential
from keras.layers import Conv1D, ConvLSTM2D, MaxPooling1D, Flatten, Dense, Reshape,TimeDistributed,Conv2D,BatchNormalization,MaxPooling2D,ConvLSTM1D,LSTM



In [3]:
import tensorflow as tf
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)


In [4]:
# Read CSV File
df = pd.read_csv(r"C:\Users\mujta\OneDrive\Documents\GitHub\F20PA-DriverDrowsinessDetection\Experiments\mergedEMYA_min_max_normEMYA.csv")

In [5]:
# Select frames at 5 fps
df = df.iloc[::2]


In [6]:
df.drop(columns=["Unnamed: 0","Angle","Part_ID"],inplace=True)

In [7]:
df

,Label,EAR,MAR,MOE
0,0,0.695510,0.616864,0.391060
2,0,0.700934,0.642625,0.404756
4,0,0.707608,0.535723,0.335285
6,0,0.752346,0.612585,0.364306
8,0,0.692796,0.607403,0.386318
...,...,...,...,...
422990,10,0.491076,0.604703,0.363197
422992,10,0.523784,0.654568,0.354916
422994,10,0.571002,0.590067,0.304906
422996,10,0.471807,0.654889,0.392917


In [8]:
int(df.shape[0]/5)

42300

In [9]:

from sklearn.metrics import precision_recall_fscore_support
# Function to get precision, recall, and F1 scores

def getPerformanceMetrics(model,X_test,y_test):

    y_pred_proba = model.predict(X_test)
    y_pred = np.argmax(y_pred_proba, axis=1)
    
    # Compute precision, recall, and F1 score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(y_test.flatten(), y_pred, average=None)
    # lists of precision , recall, and F1 scores
    p_list = []
    r_list = []
    f_list = []
    # Print precision, recall, and F1 score for each class
    for i in range(len(precision)):
        p_list.append(precision[i])
        r_list.append(recall[i])
        f_list.append(f1[i])
        print(f"Class {i}:")
        print(f"  Precision: {precision[i]}")
        print(f"  Recall: {recall[i]}")
        print(f"  F1 Score: {f1[i]}")
    # Print Average values
    print(f"Avg Precision:{np.average(p_list)}")
    print(f"Avg Recall:{np.average(r_list)}")
    print(f"Avg F1 Score:{np.average(f_list)}")



In [10]:
# Function to set parameters and train model.
# Returns model, and test sets.
def compileModel(data,timestep,test_size,num_epochs):
    # Separate labels
    y=data["Label"]
    data = data.drop(columns=['Label'])
    # Reshape according to timestep
    df_reshaped = np.array(data).reshape(int(data.shape[0]/timestep),timestep,3)
    y = np.array(y)
    y_reshaped  = []
    for i in range(0, len(y), timestep):
        y_reshaped.append([y[i]])
    # Create test and  train sets.
    X_train, X_test, y_train, y_test = train_test_split(df_reshaped, y_reshaped, test_size=test_size, random_state=42)

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train= np.array(y_train)
    y_test = np.array(y_test)
    # Set labels as  0,1,2
    for i in range(0,len(y_train)):
        y_train[i]=int(y_train[i]/5)
    for i in range(0,len(y_test)):
        y_test[i]=int(y_test[i]/5)
    # Define model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(timestep, 3)))
    model.add(MaxPooling1D(pool_size=2))

    model.add(LSTM(64, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(3, activation='softmax'))  

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=num_epochs, batch_size=32, validation_split=0.2)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy:", accuracy)
    getPerformanceMetrics(model,X_test,y_test)
    return model, X_test, y_test






## Increase timestep 5,10,20

In [11]:
modelA , X_testA, y_testA = compileModel(data=df,
                                         timestep=5,
                                         test_size=0.2,
                                         num_epochs=50)




Epoch 1/50


846/846 [==============================] - 6s 4ms/step - loss: 1.0629 - accuracy: 0.4164 - val_loss: 1.0460 - val_accuracy: 0.4309
Epoch 2/50
846/846 [==============================] - 3s 4ms/step - loss: 1.0394 - accuracy: 0.4359 - val_loss: 1.0389 - val_accuracy: 0.4301
Epoch 3/50
846/846 [==============================] - 3s 4ms/step - loss: 1.0287 - accuracy: 0.4509 - val_loss: 1.0300 - val_accuracy: 0.4511
Epoch 4/50
846/846 [==============================] - 3s 3ms/step - loss: 1.0127 - accuracy: 0.4625 - val_loss: 1.0124 - val_accuracy: 0.4657
Epoch 5/50
846/846 [==============================] - 3s 4ms/step - loss: 1.0009 - accuracy: 0.4761 - val_loss: 1.0030 - val_accuracy: 0.4750
Epoch 6/50
846/846 [==============================] - 3s 4ms/step - loss: 0.9886 - accuracy: 0.4890 - val_loss: 0.9895 - val_accuracy: 0.4900
Epoch 7/50
846/846 [==============================] - 3s 4ms/step - loss: 0.9804 - accuracy: 0.4979 - val_loss: 0.9815 - val_accuracy: 0.4967
E

In [12]:
modelA.save("modelA.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
modelB , X_testB, y_testB = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.2,
                                         num_epochs=50)

Epoch 1/50
423/423 [==============================] - 7s 8ms/step - loss: 1.0659 - accuracy: 0.4145 - val_loss: 1.0425 - val_accuracy: 0.4137
Epoch 2/50
423/423 [==============================] - 2s 5ms/step - loss: 1.0404 - accuracy: 0.4403 - val_loss: 1.0307 - val_accuracy: 0.4486
Epoch 3/50
423/423 [==============================] - 2s 5ms/step - loss: 1.0259 - accuracy: 0.4532 - val_loss: 1.0145 - val_accuracy: 0.4654
Epoch 4/50
423/423 [==============================] - 2s 5ms/step - loss: 1.0161 - accuracy: 0.4682 - val_loss: 0.9988 - val_accuracy: 0.4737
Epoch 5/50
423/423 [==============================] - 2s 5ms/step - loss: 0.9983 - accuracy: 0.4796 - val_loss: 0.9811 - val_accuracy: 0.4814
Epoch 6/50
423/423 [==============================] - 2s 5ms/step - loss: 0.9843 - accuracy: 0.5072 - val_loss: 0.9624 - val_accuracy: 0.5313
Epoch 7/50
423/423 [==============================] - 2s 6ms/step - loss: 0.9658 - accuracy: 0.5230 - val_loss: 0.9496 - val_accuracy: 0.5260
Epoch 

In [14]:
modelB.save("modelB.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
modelC , X_testC, y_testC = compileModel(data=df,
                                         timestep=20,
                                         test_size=0.2,
                                         num_epochs=50)

Epoch 1/50
212/212 [==============================] - 7s 13ms/step - loss: 1.0658 - accuracy: 0.4097 - val_loss: 1.0411 - val_accuracy: 0.4362
Epoch 2/50
212/212 [==============================] - 2s 9ms/step - loss: 1.0395 - accuracy: 0.4303 - val_loss: 1.0343 - val_accuracy: 0.4362
Epoch 3/50
212/212 [==============================] - 2s 9ms/step - loss: 1.0281 - accuracy: 0.4419 - val_loss: 1.0271 - val_accuracy: 0.4462
Epoch 4/50
212/212 [==============================] - 2s 9ms/step - loss: 1.0181 - accuracy: 0.4564 - val_loss: 1.0231 - val_accuracy: 0.4450
Epoch 5/50
212/212 [==============================] - 2s 9ms/step - loss: 1.0108 - accuracy: 0.4573 - val_loss: 1.0082 - val_accuracy: 0.4746
Epoch 6/50
212/212 [==============================] - 2s 9ms/step - loss: 1.0083 - accuracy: 0.4727 - val_loss: 1.0019 - val_accuracy: 0.4693
Epoch 7/50
212/212 [==============================] - 2s 8ms/step - loss: 0.9992 - accuracy: 0.4811 - val_loss: 0.9925 - val_accuracy: 0.4775
Epoch

In [16]:
modelC.save("modelC.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Since Model B has best performance use that to experiment with num epochs

In [17]:
modelD , X_testD, y_testD = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.2,
                                         num_epochs=75)

Epoch 1/75
423/423 [==============================] - 6s 7ms/step - loss: 1.0663 - accuracy: 0.4105 - val_loss: 1.0487 - val_accuracy: 0.4075
Epoch 2/75
423/423 [==============================] - 2s 5ms/step - loss: 1.0402 - accuracy: 0.4407 - val_loss: 1.0320 - val_accuracy: 0.4542
Epoch 3/75
423/423 [==============================] - 2s 5ms/step - loss: 1.0238 - accuracy: 0.4560 - val_loss: 1.0060 - val_accuracy: 0.4648
Epoch 4/75
423/423 [==============================] - 2s 5ms/step - loss: 1.0136 - accuracy: 0.4692 - val_loss: 0.9840 - val_accuracy: 0.4970
Epoch 5/75
423/423 [==============================] - 2s 5ms/step - loss: 0.9962 - accuracy: 0.4832 - val_loss: 0.9755 - val_accuracy: 0.5000
Epoch 6/75
423/423 [==============================] - 2s 5ms/step - loss: 0.9862 - accuracy: 0.5004 - val_loss: 0.9769 - val_accuracy: 0.4840
Epoch 7/75
423/423 [==============================] - 2s 5ms/step - loss: 0.9696 - accuracy: 0.5147 - val_loss: 0.9473 - val_accuracy: 0.5228
Epoch 

In [18]:
modelD.save("modelD.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
modelE , X_testE, y_testE = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.2,
                                         num_epochs=100)

Epoch 1/100
423/423 [==============================] - 6s 6ms/step - loss: 1.0651 - accuracy: 0.4164 - val_loss: 1.0418 - val_accuracy: 0.4574
Epoch 2/100
423/423 [==============================] - 2s 5ms/step - loss: 1.0360 - accuracy: 0.4457 - val_loss: 1.0250 - val_accuracy: 0.4495
Epoch 3/100
423/423 [==============================] - 2s 5ms/step - loss: 1.0231 - accuracy: 0.4563 - val_loss: 1.0048 - val_accuracy: 0.4713
Epoch 4/100
423/423 [==============================] - 2s 5ms/step - loss: 1.0112 - accuracy: 0.4713 - val_loss: 0.9859 - val_accuracy: 0.4843
Epoch 5/100
423/423 [==============================] - 2s 4ms/step - loss: 0.9998 - accuracy: 0.4772 - val_loss: 0.9760 - val_accuracy: 0.4902
Epoch 6/100
423/423 [==============================] - 2s 4ms/step - loss: 0.9790 - accuracy: 0.5118 - val_loss: 0.9656 - val_accuracy: 0.5044
Epoch 7/100
423/423 [==============================] - 2s 5ms/step - loss: 0.9584 - accuracy: 0.5288 - val_loss: 0.9370 - val_accuracy: 0.5346

In [20]:
modelE.save("modelE.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Use best performance and experiment with test size

### 70-30 split

In [21]:
modelF , X_testF, y_testF = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.3,
                                         num_epochs=100)

Epoch 1/100
371/371 [==============================] - 6s 7ms/step - loss: 1.0647 - accuracy: 0.4130 - val_loss: 1.0323 - val_accuracy: 0.4634
Epoch 2/100
371/371 [==============================] - 2s 5ms/step - loss: 1.0403 - accuracy: 0.4429 - val_loss: 1.0183 - val_accuracy: 0.4613
Epoch 3/100
371/371 [==============================] - 2s 5ms/step - loss: 1.0316 - accuracy: 0.4482 - val_loss: 1.0130 - val_accuracy: 0.4637
Epoch 4/100
371/371 [==============================] - 2s 5ms/step - loss: 1.0203 - accuracy: 0.4563 - val_loss: 0.9916 - val_accuracy: 0.4677
Epoch 5/100
371/371 [==============================] - 2s 5ms/step - loss: 1.0112 - accuracy: 0.4618 - val_loss: 0.9838 - val_accuracy: 0.4995
Epoch 6/100
371/371 [==============================] - 2s 5ms/step - loss: 0.9980 - accuracy: 0.4823 - val_loss: 1.0003 - val_accuracy: 0.4802
Epoch 7/100
371/371 [==============================] - 2s 6ms/step - loss: 0.9883 - accuracy: 0.4921 - val_loss: 0.9957 - val_accuracy: 0.4735

In [22]:
modelF.save("modelF.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### 50-50 split

In [23]:
modelG , X_testG, y_testG = compileModel(data=df,
                                         timestep=10,
                                         test_size=0.5,
                                         num_epochs=100)

Epoch 1/100
265/265 [==============================] - 5s 8ms/step - loss: 1.0736 - accuracy: 0.4045 - val_loss: 1.0434 - val_accuracy: 0.4596
Epoch 2/100
265/265 [==============================] - 1s 6ms/step - loss: 1.0438 - accuracy: 0.4408 - val_loss: 1.0340 - val_accuracy: 0.4596
Epoch 3/100
265/265 [==============================] - 2s 6ms/step - loss: 1.0318 - accuracy: 0.4482 - val_loss: 1.0207 - val_accuracy: 0.4591
Epoch 4/100
265/265 [==============================] - 2s 7ms/step - loss: 1.0292 - accuracy: 0.4453 - val_loss: 1.0225 - val_accuracy: 0.4652
Epoch 5/100
265/265 [==============================] - 1s 5ms/step - loss: 1.0222 - accuracy: 0.4576 - val_loss: 1.0047 - val_accuracy: 0.4870
Epoch 6/100
265/265 [==============================] - 2s 6ms/step - loss: 1.0123 - accuracy: 0.4741 - val_loss: 0.9997 - val_accuracy: 0.4846
Epoch 7/100
265/265 [==============================] - 1s 5ms/step - loss: 1.0087 - accuracy: 0.4801 - val_loss: 0.9873 - val_accuracy: 0.5087

In [24]:
modelG.save("modelG.h5")

c:\Users\mujta\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
